# Baseline (T-F)

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import tensorflow as tf
import tensorflow_addons as tfa
from keras import layers

import librosa
from audiomentations import SpecCompose, SpecFrequencyMask

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import pickle
import argparse
import wandb
from wandb.keras import WandbCallback
wandb.init(project="DACON_235910", name="Baseline")

parser = argparse.ArgumentParser(description="Baseline")
parser.add_argument('--feature', default="mfcc", type=str) # mfcc or melspec
parser.add_argument('--cnn_model', default="2d", type=str) # 1d or 2d
parser.add_argument('--sampling_rate', default=16000, type=int)
# parser.add_argument('--frequency_mask', default=0, type=float)
parser.add_argument('--optimizer', default="sgd", type=str) # sgd or adam
parser.add_argument('--loss', default="fl", type=str) # bc or fl
parser.add_argument('--learning_rate', default=0.001, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--cv', default=10, type=int)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

feature = args.feature
cnn_model = args.cnn_model
sampling_rate = args.sampling_rate
# frequency_mask = args.frequency_mask
optimizer = args.optimizer
loss = args.loss
learning_rate = args.learning_rate
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
cv = args.cv
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

# augment = SpecCompose([SpecFrequencyMask(p=frequency_mask)])
    
with open('data/train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)
with open('data/test_df.pkl', 'rb') as f:
    test_df = pickle.load(f)
    
train_df.head()

wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,data,id,age,gender,respiratory_condition,fever_or_muscle_pain,covid19
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,24,female,0,1,0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2,51,male,0,0,0
2,"[2.7372453e-09, -1.0615647e-08, 5.2142607e-08,...",3,22,male,0,0,0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4,29,female,1,0,0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5,23,male,0,0,0


In [2]:
train_df["covid19"].value_counts(normalize=True)

0    0.91958
1    0.08042
Name: covid19, dtype: float64

## Data Generation

In [3]:
# train_df = pd.read_csv("data/train_data.csv")
# test_df = pd.read_csv("data/test_data.csv")

# train_folder = "data/train/"
# test_folder = "data/test/"

# def dataset(folder, df):
#     dataset = []
#     for uid in tqdm(df['id']):
#         path = os.path.join(folder, str(uid).zfill(5)+'.wav')
#         y, sr = librosa.load(path, sr=sampling_rate)
#         y = librosa.util.normalize(y)
#         dataset.append([y])
#     dataset = pd.DataFrame(dataset, columns=['data'])
#     dataset = pd.concat([dataset, df], axis=1)
#     return dataset

# train_df = dataset(train_folder, train_df)
# test_df = dataset(test_folder, test_df)

# with open('train_df.pkl', 'wb') as f:
#     pickle.dump(train_df, f, pickle.HIGHEST_PROTOCOL)
# with open('test_df.pkl', 'wb') as f:
#     pickle.dump(test_df, f, pickle.HIGHEST_PROTOCOL)

## Preprocessing

In [4]:
def preprocess_dataset(data):

    frame_length = 0.025
    frame_stride = 0.010

    input_nfft = int(round(sampling_rate*frame_length))
    input_stride = int(round(sampling_rate*frame_stride))

    extracted_features = []
    for i in tqdm(data):
        if feature == "mfcc":
            n_feature = 40
            S = librosa.feature.mfcc(y=i,
                                     sr=sampling_rate,
                                     n_mfcc=n_feature,
                                     n_fft=input_nfft,
                                     hop_length=input_stride)
        elif feature == "melspec":
            n_feature = 128
            S = librosa.feature.melspectrogram(y=i,
                                               sr=sampling_rate,
                                               n_mels=n_feature,
                                               n_fft=input_nfft,
                                               hop_length=input_stride)
            S = librosa.power_to_db(S, ref=np.max)
        S = tf.image.resize(S[:, :, np.newaxis], (n_feature, n_feature))
        if cnn_model == "1d":
            S = np.transpose(S[:,:,0], (0, 1))
        extracted_features.append(S)
    return extracted_features

X = np.array(preprocess_dataset(train_df["data"]))
X_test = np.array(preprocess_dataset(test_df["data"]))
y = train_df["covid19"]

X.shape, y.shape, X_test.shape

100%|██████████████████████████████████████████████████████████████████████████████| 5732/5732 [01:11<00:00, 80.44it/s]


((3805, 40, 40, 1), (3805,), (5732, 40, 40, 1))

In [5]:
def preprocess_feature(df):
    temp = df.copy()
    temp["condition1"] = temp["respiratory_condition"] + temp["fever_or_muscle_pain"]
    temp["condition2"] = temp["respiratory_condition"] * temp["fever_or_muscle_pain"]
    temp = temp.drop(["id", "age", "gender", "respiratory_condition", "fever_or_muscle_pain"], axis=1)
    return temp

train_df = preprocess_feature(train_df)
test_df = preprocess_feature(test_df)

X_test_tab = test_df[["condition1", "condition2"]].values

X_test_tab.shape

(5732, 2)

## Training

In [6]:
def scratch_1dcnn():
    
    inp = tf.keras.Input(shape=(X.shape[1], X.shape[2]))
    tab = tf.keras.Input(shape=(X_test_tab.shape[1],))
    
    x = layers.Conv1D(256, 3, 1, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(256, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D()(x)
    
    x = layers.Conv1D(512, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(512, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = layers.Conv1D(1024, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(1024, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Concatenate()([x, tab])
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    
    oup = layers.Dense(1, activation="sigmoid")(x)
    
    model = tf.keras.Model(inputs=[inp, tab], outputs=oup)
    
    return model


def scratch_2dcnn():
    
    inp = tf.keras.Input(shape=(X.shape[1], X.shape[2], 1))
    tab = tf.keras.Input(shape=(X_test_tab.shape[1],))
    
    x = layers.Conv2D(64, 3, 1, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(128, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(128, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(256, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(256, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Concatenate()([x, tab])
    
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    
    oup = layers.Dense(1, activation="sigmoid")(x)
    
    model = tf.keras.Model(inputs=[inp, tab], outputs=oup)
    
    return model

if cnn_model == "1d":
    model = scratch_1dcnn()
elif cnn_model == "2d":
    model = scratch_2dcnn()
    
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 40, 40, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 40, 40, 64)   640         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 40, 40, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 40, 40, 64)   0           ['batch_normalization[0][0]']

In [7]:
validations = []
thresholds = []
predictions = []

idx=0

skf = StratifiedKFold(n_splits=cv)
for train_index, val_index in tqdm(skf.split(X, y)):
    
    idx+=1

    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    X_train_tab = train_df[["condition1", "condition2"]].values[train_index]
    X_val_tab = train_df[["condition1", "condition2"]].values[val_index]
    
    train_ds = (
        tf.data.Dataset.from_tensor_slices(((X_train, X_train_tab), y_train))
        .shuffle(len(X_train))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    val_ds = (
        tf.data.Dataset.from_tensor_slices(((X_val, X_val_tab), y_val))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    if cnn_model == "1d":
        model = scratch_1dcnn()
    elif cnn_model == "2d":
        model = scratch_2dcnn()

    lr = tf.keras.optimizers.schedules.CosineDecay(learning_rate, decay_steps=1000)
    if optimizer == "adam":
        optim = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
        
    if loss == "bc":
        label_smoothing=0
        loss_function = tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing)
    elif loss == "fl":
        loss_function = tfa.losses.SigmoidFocalCrossEntropy()

    model.compile(
        optimizer=optim,
        loss=loss_function,
    )
    
    checkpoint_filepath=f"load_model/{parser.description}_{idx}"

    checkpoint_callback = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint_callback, WandbCallback()],
    )

    max_f1 = 0
    threshold = 0
    for temp_threshold in np.linspace(0.05, 0.95, 19):
        temp_f1 = f1_score(y_val, np.where(model.predict(val_ds)>temp_threshold, 1, 0), average="macro")
        if temp_f1 > max_f1:
            max_f1 = temp_f1
            threshold = temp_threshold
    
    print(f"idx:{idx}, f1:{max_f1}, threshold:{threshold}")
    
    validations.append(max_f1)
    thresholds.append(threshold)
    predictions.append(model.predict([X_test, X_test_tab]))

val_f1 = np.mean(validations, axis=0)

print("validation_macro-f1: ", val_f1)
wandb.log({'validation_macro-f1': val_f1})

0it [00:00, ?it/s]

Epoch 1/100
107/107 [==============================] - 9s 28ms/step - loss: 0.0335 - val_loss: 0.0328 - _timestamp: 1655737024.0000 - _runtime: 145.0000
Epoch 2/100
107/107 [==============================] - 3s 25ms/step - loss: 0.0287 - val_loss: 0.0301 - _timestamp: 1655737027.0000 - _runtime: 148.0000
Epoch 3/100
107/107 [==============================] - 3s 25ms/step - loss: 0.0269 - val_loss: 0.0263 - _timestamp: 1655737030.0000 - _runtime: 151.0000
Epoch 4/100
107/107 [==============================] - 3s 24ms/step - loss: 0.0263 - val_loss: 0.0262 - _timestamp: 1655737032.0000 - _runtime: 153.0000
Epoch 5/100
107/107 [==============================] - 3s 25ms/step - loss: 0.0261 - val_loss: 0.0261 - _timestamp: 1655737035.0000 - _runtime: 156.0000
Epoch 6/100
107/107 [==============================] - 2s 22ms/step - loss: 0.0257 - val_loss: 0.0262 - _timestamp: 1655737038.0000 - _runtime: 159.0000
Epoch 7/100
107/107 [==============================] - 3s 25ms/step - loss: 0.0254

1it [00:57, 57.02s/it]

Epoch 1/100
107/107 [==============================] - 4s 26ms/step - loss: 0.0313 - val_loss: 0.0399 - _timestamp: 1655737076.0000 - _runtime: 197.0000
Epoch 2/100
107/107 [==============================] - 3s 25ms/step - loss: 0.0276 - val_loss: 0.0253 - _timestamp: 1655737079.0000 - _runtime: 200.0000
Epoch 3/100
107/107 [==============================] - 3s 26ms/step - loss: 0.0267 - val_loss: 0.0242 - _timestamp: 1655737082.0000 - _runtime: 203.0000
Epoch 4/100
107/107 [==============================] - 3s 26ms/step - loss: 0.0263 - val_loss: 0.0238 - _timestamp: 1655737084.0000 - _runtime: 205.0000
Epoch 5/100
107/107 [==============================] - 2s 23ms/step - loss: 0.0256 - val_loss: 0.0242 - _timestamp: 1655737087.0000 - _runtime: 208.0000
Epoch 6/100
107/107 [==============================] - 3s 26ms/step - loss: 0.0250 - val_loss: 0.0236 - _timestamp: 1655737090.0000 - _runtime: 211.0000
Epoch 7/100
107/107 [==============================] - 3s 27ms/step - loss: 0.0247

2it [01:52, 55.89s/it]

...

Epoch 1/100
108/108 [==============================] - 4s 26ms/step - loss: 0.0333 - val_loss: 0.0273 - _timestamp: 1655737446.0000 - _runtime: 567.0000
Epoch 2/100
108/108 [==============================] - 3s 24ms/step - loss: 0.0275 - val_loss: 0.0253 - _timestamp: 1655737449.0000 - _runtime: 570.0000
Epoch 3/100
108/108 [==============================] - 2s 22ms/step - loss: 0.0262 - val_loss: 0.0254 - _timestamp: 1655737451.0000 - _runtime: 572.0000
Epoch 4/100
108/108 [==============================] - 3s 23ms/step - loss: 0.0262 - val_loss: 0.0246 - _timestamp: 1655737454.0000 - _runtime: 575.0000
Epoch 5/100
108/108 [==============================] - 2s 22ms/step - loss: 0.0257 - val_loss: 0.0255 - _timestamp: 1655737456.0000 - _runtime: 577.0000
Epoch 6/100
108/108 [==============================] - 2s 22ms/step - loss: 0.0252 - val_loss: 0.0252 - _timestamp: 1655737459.0000 - _runtime: 580.0000
Epoch 7/100
108/108 [==============================] - 2s 22ms/step - loss: 0.0250

9it [07:47, 49.09s/it]

Epoch 1/100
108/108 [==============================] - 4s 26ms/step - loss: 0.0350 - val_loss: 0.0318 - _timestamp: 1655737487.0000 - _runtime: 608.0000
Epoch 2/100
108/108 [==============================] - 3s 25ms/step - loss: 0.0277 - val_loss: 0.0260 - _timestamp: 1655737490.0000 - _runtime: 611.0000
Epoch 3/100
108/108 [==============================] - 3s 25ms/step - loss: 0.0273 - val_loss: 0.0256 - _timestamp: 1655737492.0000 - _runtime: 613.0000
Epoch 4/100
108/108 [==============================] - 3s 23ms/step - loss: 0.0262 - val_loss: 0.0254 - _timestamp: 1655737495.0000 - _runtime: 616.0000
Epoch 5/100
108/108 [==============================] - 3s 24ms/step - loss: 0.0256 - val_loss: 0.0256 - _timestamp: 1655737498.0000 - _runtime: 619.0000
Epoch 6/100
108/108 [==============================] - 3s 25ms/step - loss: 0.0255 - val_loss: 0.0250 - _timestamp: 1655737500.0000 - _runtime: 621.0000
Epoch 7/100
108/108 [==============================] - 3s 24ms/step - loss: 0.0251

10it [08:50, 53.05s/it]

validation_macro-f1:  0.6066403676408889


## Inference

In [8]:
threshold = np.mean(thresholds)
test_df["covid19"] = np.where(np.mean(predictions, axis=0)>threshold, 1, 0)

submission = pd.read_csv('data/sample_submission.csv')
submission['covid19'] = test_df['covid19']
submission.to_csv('submission.csv', index=False)

test_df['covid19'].value_counts(normalize=True)

0    0.908932
1    0.091068
Name: covid19, dtype: float64